In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf

df = pd.read_csv(r"C:\Users\m\Taha\Codebasics Challenges\Parliament Elections\LOK SABHA.csv")

In [2]:
df.head()

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors,Year
0,Andhra Pradesh,Adilabad,Godam Nagesh,Male,49,ST,TRS,Car,425762,5085,430847,1386282,2014
1,Andhra Pradesh,Adilabad,Naresh,Male,37,ST,INC,Hand,257994,1563,259557,1386282,2014
2,Andhra Pradesh,Adilabad,Ramesh Rathod,Male,48,ST,TDP,Bicycle,182879,1319,184198,1386282,2014
3,Andhra Pradesh,Adilabad,Rathod Sadashiv,Male,55,ST,BSP,Elephant,94363,57,94420,1386282,2014
4,Andhra Pradesh,Adilabad,Nethawath Ramdas,Male,44,ST,IND,Auto- Rickshaw,41028,4,41032,1386282,2014


In [ ]:
query = """
select 
    party,
    Year,
    count(*) as seats
from
    winners_parties
group by
    party, Year
order by
    seats desc
"""
sqldf(query)

## 6. % split of votes of parties between 14 vs 19 at national level

In [22]:
query = """
select 
    pc_name,
    party,
    sum(case when Year = 2014 then total_votes else 0 end) as party_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as party_2019_votes
from 
    df
group by 
    pc_name, party
"""

party_votes = sqldf(query)
party_votes.head()

,pc_name,party,party_2014_votes,party_2019_votes
0,Adilabad,APOI,0,6837
1,Adilabad,BJP,0,377374
2,Adilabad,BSP,94420,0
3,Adilabad,INC,259557,314238
4,Adilabad,IND,59733,17478


In [23]:
query = """
select 
    pc_name,
    sum(case when Year = 2014 then total_votes else 0 end) as pc_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as pc_2019_votes
from
    df
group by
    pc_name

"""
pc_votes = sqldf(query)
pc_votes.head()

,pc_name,pc_2014_votes,pc_2019_votes
0,Adilabad,1045839,1063730
1,Agra,1070400,1145323
2,Ahmadnagar,1062318,1203797
3,Ahmedabad East,985525,1116367
4,Ahmedabad West,964609,997024


In [24]:
national_level = pd.merge(party_votes, pc_votes, on = 'pc_name')
national_level 

,pc_name,party,party_2014_votes,party_2019_votes,pc_2014_votes,pc_2019_votes
0,Adilabad,APOI,0,6837,1045839,1063730
1,Adilabad,BJP,0,377374,1045839,1063730
2,Adilabad,BSP,94420,0,1045839,1063730
3,Adilabad,INC,259557,314238,1045839,1063730
4,Adilabad,IND,59733,17478,1045839,1063730
...,...,...,...,...,...,...
8504,Zahirabad,PPOI,0,1279,1094806,1044365
8505,Zahirabad,RPI(A),18027,0,1094806,1044365
8506,Zahirabad,TDP,157497,0,1094806,1044365
8507,Zahirabad,TRS,508661,434244,1094806,1044365


In [25]:
query = """
select 
    *
from
(
select 
    *,
    row_number() over(partition by pc_name order by party_2014_perc desc) as party_rank_2014,
    row_number() over(partition by pc_name order by party_2019_perc desc) as party_rank_2019
from
(
select
    pc_name,
    pc_2014_votes,
    pc_2019_votes,
    party,
    round(((cast(party_2014_votes as real)/pc_2014_votes) * 100),2) as party_2014_perc,
    round(((cast(party_2019_votes as real)/pc_2019_votes) * 100),2) as party_2019_perc
from
    national_level
                    ) as sub
                    ) as sub1

"""
Q6 = sqldf(query)
Q6

,pc_name,pc_2014_votes,pc_2019_votes,party,party_2014_perc,party_2019_perc,party_rank_2014,party_rank_2019
0,Adilabad,1045839,1063730,TRS,41.20,29.97,1,2
1,Adilabad,1045839,1063730,INC,24.82,29.54,2,3
2,Adilabad,1045839,1063730,TDP,17.61,0.00,3,11
3,Adilabad,1045839,1063730,BSP,9.03,0.00,4,10
4,Adilabad,1045839,1063730,IND,5.71,1.64,5,4
...,...,...,...,...,...,...,...,...
8504,Zahirabad,1094806,1044365,BARP,0.00,0.61,11,6
8505,Zahirabad,1094806,1044365,BMUP,0.00,0.61,12,7
8506,Zahirabad,1094806,1044365,ANC,0.00,0.16,13,8
8507,Zahirabad,1094806,1044365,IPBP,0.00,0.15,14,9


In [26]:
seats_2019 = Q6[(Q6['party_rank_2019'] == 1) & (Q6['pc_2019_votes'] != 0)]['party'].value_counts().reset_index()

In [27]:
seats_2014 = Q6[(Q6['party_rank_2014'] == 1) & (Q6['pc_2014_votes'] != 0)]['party'].value_counts().reset_index()

In [77]:
split_by_constituencies = pd.merge(seats_2014, seats_2019, on = 'party', how = 'outer').fillna(0)
split_by_constituencies.rename(columns={'count_x': '2014 seats', 'count_y': '2019 seats'}, inplace=True)
split_by_constituencies['seats diff'] = split_by_constituencies['2019 seats'] - split_by_constituencies['2014 seats']

In [79]:
split_by_constituencies[split_by_constituencies['seats diff']>0]

,party,2014 seats,2019 seats,seats diff
0,BJP,267.0,300.0,33.0
1,INC,44.0,52.0,8.0
4,SHS,17.0,18.0,1.0
7,YSRCP,9.0,22.0,13.0
14,IND,4.0,6.0,2.0
19,JD(U),2.0,16.0,14.0
21,IUML,2.0,3.0,1.0
33,CPI,1.0,2.0,1.0
35,DMK,0.0,24.0,24.0
36,BJD,0.0,12.0,12.0


In [80]:
split_by_constituencies[split_by_constituencies['seats diff']<0]

,party,2014 seats,2019 seats,seats diff
2,ADMK,37.0,1.0,-36.0
3,AITC,34.0,22.0,-12.0
5,TDP,16.0,3.0,-13.0
6,TRS,11.0,9.0,-2.0
8,CPM,9.0,0.0,-9.0
9,NCP,6.0,5.0,-1.0
12,SAD,4.0,2.0,-2.0
13,RJD,4.0,0.0,-4.0
15,AAAP,4.0,1.0,-3.0
16,JKPDP,3.0,0.0,-3.0


## 7. % split of votes of parties between 14 vs 19 at state level

In [52]:
query = """
select 
    state,
    party,
    sum(case when Year = 2014 then total_votes else 0 end) as party_state_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as party_state_2019_votes
from 
    df
group by 
    state, party
"""

party_state_votes = sqldf(query)

In [53]:
query = """
select 
    state,
    sum(case when Year = 2014 then total_votes else 0 end) as state_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as state_2019_votes
from
    df
group by
    state

"""
state_votes = sqldf(query)

In [54]:
state_level = pd.merge(party_state_votes, state_votes, on = 'state')
state_level

,state,party,party_state_2014_votes,party_state_2019_votes,state_2014_votes,state_2019_votes
0,Andaman & Nicobar Islands,AAAP,3737,2839,190328,207296
1,Andaman & Nicobar Islands,AIFB,225,0,190328,207296
2,Andaman & Nicobar Islands,AINHCP,0,212,190328,207296
3,Andaman & Nicobar Islands,AITC,2283,1721,190328,207296
4,Andaman & Nicobar Islands,BJP,90969,93901,190328,207296
...,...,...,...,...,...,...
1940,West Bengal,SUCI(C),0,217376,51631828,57206976
1941,West Bengal,SWJP,5340,1899,51631828,57206976
1942,West Bengal,TRMRPPI,1071,0,51631828,57206976
1943,West Bengal,UTSAP,0,2375,51631828,57206976


In [57]:
query = """
select 
    *,
    party_state_2019_perc - party_state_2014_perc as perc_diff,
    row_number() over(partition by state order by party_state_2014_perc desc) as rank_2014,
    row_number() over(partition by state order by party_state_2019_perc desc) as rank_2019
from
(select
    state,
    party,
    round(((cast(party_state_2014_votes as real)/state_2014_votes) * 100),2) as party_state_2014_perc,
    round(((cast(party_state_2019_votes as real)/state_2019_votes) * 100),2) as party_state_2019_perc
from
    state_level) as sub
    
order by
    perc_diff desc
"""
Q7 = sqldf(query)
Q7

,state,party,party_state_2014_perc,party_state_2019_perc,perc_diff,rank_2014,rank_2019
0,Nagaland,NDPP,0.00,49.73,49.73,5,1
1,Dadra & Nagar Haveli,IND,1.69,48.98,47.29,4,1
2,Mizoram,MNF,0.00,44.89,44.89,5,2
3,Tripura,BJP,5.70,49.03,43.33,4,1
4,Puducherry,INC,26.35,56.27,29.92,2,1
...,...,...,...,...,...,...,...
1940,Telangana,TSP,NaN,0.01,NaN,42,42
1941,Telangana,YSRPP,NaN,0.01,NaN,43,43
1942,Telangana,ABML(S),NaN,0.00,NaN,44,44
1943,Telangana,PRSTP,NaN,0.00,NaN,45,45


In [69]:
state_seats_2014 = Q7[(Q7['rank_2014'] == 1) & (Q7['party_state_2014_perc'] !=0)]['party'].value_counts().reset_index()
state_seats_2019 = Q7[(Q7['rank_2019'] == 1) & (Q7['party_state_2019_perc'] !=0)]['party'].value_counts().reset_index()

split_by_states = pd.merge(state_seats_2014,state_seats_2019, on = 'party', how = 'outer').fillna(0)
split_by_states.rename(columns={'count_x': '2014 state seats', 'count_y': '2019 state seats'}, inplace=True)

In [76]:
split_by_states.head(5)

,party,2014 state seats,2019 state seats
0,BJP,21.0,21.0
1,INC,5.0,5.0
2,TDP,1.0,0.0
3,AITC,1.0,1.0
4,NCP,1.0,1.0


In [ ]:
## 6. % split of votes of parties between 14 vs 19 at national level

query = """
select 
    pc_name,
    party,
    sum(case when Year = 2014 then total_votes else 0 end) as party_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as party_2019_votes
from 
    df
group by 
    pc_name, party
"""

party_votes = sqldf(query)
party_votes.head()

query = """
select 
    pc_name,
    sum(case when Year = 2014 then total_votes else 0 end) as pc_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as pc_2019_votes
from
    df
group by
    pc_name

"""
pc_votes = sqldf(query)
pc_votes.head()

national_level = pd.merge(party_votes, pc_votes, on = 'pc_name')
national_level 

query = """
select 
    *
from
(
select 
    *,
    row_number() over(partition by pc_name order by party_2014_perc desc) as party_rank_2014,
    row_number() over(partition by pc_name order by party_2019_perc desc) as party_rank_2019
from
(
select
    pc_name,
    pc_2014_votes,
    pc_2019_votes,
    party,
    round(((cast(party_2014_votes as real)/pc_2014_votes) * 100),2) as party_2014_perc,
    round(((cast(party_2019_votes as real)/pc_2019_votes) * 100),2) as party_2019_perc
from
    national_level
                    ) as sub
                    ) as sub1

"""
Q6 = sqldf(query)
Q6

seats_2019 = Q6[(Q6['party_rank_2019'] == 1) & (Q6['pc_2019_votes'] != 0)]['party'].value_counts().reset_index()

seats_2014 = Q6[(Q6['party_rank_2014'] == 1) & (Q6['pc_2014_votes'] != 0)]['party'].value_counts().reset_index()

split_by_constituencies = pd.merge(seats_2014, seats_2019, on = 'party', how = 'outer').fillna(0)
split_by_constituencies.rename(columns={'count_x': '2014 seats', 'count_y': '2019 seats'}, inplace=True)
split_by_constituencies['seats diff'] = split_by_constituencies['2019 seats'] - split_by_constituencies['2014 seats']

split_by_constituencies[split_by_constituencies['seats diff']>0]

split_by_constituencies[split_by_constituencies['seats diff']<0]

## 7. % split of votes of parties between 14 vs 19 at state level

query = """
select 
    state,
    party,
    sum(case when Year = 2014 then total_votes else 0 end) as party_state_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as party_state_2019_votes
from 
    df
group by 
    state, party
"""

party_state_votes = sqldf(query)

query = """
select 
    state,
    sum(case when Year = 2014 then total_votes else 0 end) as state_2014_votes,
    sum(case when Year = 2019 then total_votes else 0 end) as state_2019_votes
from
    df
group by
    state

"""
state_votes = sqldf(query)

state_level = pd.merge(party_state_votes, state_votes, on = 'state')
state_level

query = """
select 
    *,
    party_state_2019_perc - party_state_2014_perc as perc_diff,
    row_number() over(partition by state order by party_state_2014_perc desc) as rank_2014,
    row_number() over(partition by state order by party_state_2019_perc desc) as rank_2019
from
(select
    state,
    party,
    round(((cast(party_state_2014_votes as real)/state_2014_votes) * 100),2) as party_state_2014_perc,
    round(((cast(party_state_2019_votes as real)/state_2019_votes) * 100),2) as party_state_2019_perc
from
    state_level) as sub
    
order by
    perc_diff desc
"""
Q7 = sqldf(query)
Q7

state_seats_2014 = Q7[(Q7['rank_2014'] == 1) & (Q7['party_state_2014_perc'] !=0)]['party'].value_counts().reset_index()
state_seats_2019 = Q7[(Q7['rank_2019'] == 1) & (Q7['party_state_2019_perc'] !=0)]['party'].value_counts().reset_index()

split_by_states = pd.merge(state_seats_2014,state_seats_2019, on = 'party', how = 'outer').fillna(0)
split_by_states.rename(columns={'count_x': '2014 state seats', 'count_y': '2019 state seats'}, inplace=True)

split_by_states.head(5)